In [4]:
import pandas as pd

print("Data Ingestion Pipeline Started")

Data Ingestion Pipeline Started


In [8]:
# Correct Data Ingestion paths (RUN THIS)

customers = pd.read_csv("../olist_customers_dataset.csv")
orders = pd.read_csv("../olist_orders_dataset.csv")
items = pd.read_csv("../olist_order_items_dataset.csv")
products = pd.read_csv("../olist_products_dataset.csv")
payments = pd.read_csv("../olist_order_payments_dataset.csv")
reviews = pd.read_csv("../olist_order_reviews_dataset.csv")
sellers = pd.read_csv("../olist_sellers_dataset.csv")
geolocation = pd.read_csv("../olist_geolocation_dataset.csv")
category = pd.read_csv("../product_category_name_translation.csv")

print("All datasets loaded successfully!")

customers.head()

All datasets loaded successfully!


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [6]:
import os
print(os.getcwd())

c:\Users\Sandhya Gunde\Desktop\data-ingestion-kg-pipeline\data\notebooks


In [7]:
import os

print("Files in current folder:", os.listdir("."))

Files in current folder: ['app', 'data_ingestion_pipeline.ipynb']


In [9]:
# Basic information about the datasets
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [10]:

# Check missing values in each column
customers.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [11]:
# Remove duplicate rows (if any)
customers = customers.drop_duplicates()
print("Duplicates removed (if any). new shapes:" , customers.shape)


Duplicates removed (if any). new shapes: (99441, 5)


In [12]:
# Fill missing values appropriately
customers["customer_city"] = customers["customer_city"].fillna("Unknown")
customers["customer_state"] = customers["customer_state"].fillna("Unknown")
customers["customer_zip_code_prefix"] = customers["customer_zip_code_prefix"].fillna(0)

In [13]:
# Convert city and state to proper format
customers["customer_city"] = customers["customer_city"].str.title()
customers["customer_state"] = customers["customer_state"].str.upper()

In [14]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,Franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,Sao Bernardo Do Campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,Sao Paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,Mogi Das Cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,Campinas,SP


In [15]:
# Convert order dates to datetime format
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders["order_approved_at"] = pd.to_datetime(orders["order_approved_at"])
orders["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"])

# Fill missing delivery dates with purchase date (reasonable assumption)
orders["order_delivered_customer_date"] = orders["order_delivered_customer_date"].fillna(
    orders["order_purchase_timestamp"]
)

In [16]:
# Remove negative or zero payments (invalid data)
payments = payments[payments["payment_value"] > 0]

print("Cleaned payments shape:", payments.shape)

Cleaned payments shape: (103877, 5)


In [17]:
products["product_category_name"] = products["product_category_name"].fillna("unknown_category")

In [18]:
print("Customers null values after cleaning:\n", customers.isnull().sum())
print("\nOrders null values after cleaning:\n", orders.isnull().sum())
print("\nProducts null values after cleaning:\n", products.isnull().sum())

Customers null values after cleaning:
 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Orders null values after cleaning:
 order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date       0
order_estimated_delivery_date       0
dtype: int64

Products null values after cleaning:
 product_id                      0
product_category_name           0
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


In [19]:
# Convert all relevant order timestamps to datetime (if not already)
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])

print("Date columns standardized.")

Date columns standardized.


In [20]:
orders["delivery_time_days"] = (
    orders["order_delivered_customer_date"] -
    orders["order_purchase_timestamp"]
).dt.days

In [21]:
orders["delivery_delay_days"] = (
    orders["order_delivered_customer_date"] -
    orders["order_estimated_delivery_date"]
).dt.days

In [22]:
orders[["order_id", "delivery_time_days", "delivery_delay_days"]].head()

,order_id,delivery_time_days,delivery_delay_days
0,e481f51cbdc54678b7cc49136f2d6af7,8,-8
1,53cdb2fc8bc7dce0b6741e2150273451,13,-6
2,47770eb9100c2d0c44946d9cf07ec65d,9,-18
3,949d5b44dbf5de918fe9c16f97b45f8a,13,-13
4,ad21c59c0840e6cb83a9ceb5573f8159,2,-10


In [23]:
customers["customer_city"] = customers["customer_city"].str.title()
customers["customer_state"] = customers["customer_state"].str.upper()

In [24]:
products["product_category_name"] = (
    products["product_category_name"]
    .str.lower()
    .str.replace(" ", "_")
)

In [25]:
products["product_category_name"].head()

0               perfumaria
1                    artes
2            esporte_lazer
3                    bebes
4    utilidades_domesticas
Name: product_category_name, dtype: object

In [26]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

payments["payment_value_norm"] = scaler.fit_transform(
    payments[["payment_value"]]
)

In [27]:
payments[["payment_value", "payment_value_norm"]].head()

,payment_value,payment_value_norm
0,99.33,0.007269
1,24.39,0.001784
2,65.71,0.004808
3,107.78,0.007887
4,128.45,0.009400


In [28]:
products["product_length_norm"] = scaler.fit_transform(
    products[["product_length_cm"]]
)

products["product_height_norm"] = scaler.fit_transform(
    products[["product_height_cm"]]
)

products["product_width_norm"] = scaler.fit_transform(
    products[["product_width_cm"]]
)

In [29]:
print("Orders columns after transformation:\n", orders.columns)
print("\nPayments sample:\n", payments[["payment_value", "payment_value_norm"]].head())
print("\nProducts sample:\n", products[[
    "product_length_cm","product_length_norm",
    "product_height_cm","product_height_norm"
]].head())

Orders columns after transformation:
 Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'delivery_time_days', 'delivery_delay_days'],
      dtype='object')

Payments sample:
    payment_value  payment_value_norm
0          99.33            0.007269
1          24.39            0.001784
2          65.71            0.004808
3         107.78            0.007887
4         128.45            0.009400

Products sample:
    product_length_cm  product_length_norm  product_height_cm  \
0               16.0             0.091837               10.0   
1               30.0             0.234694               18.0   
2               18.0             0.112245                9.0   
3               26.0             0.193878                4.0   
4               20.0             0.132653               17.0   

   product_height_norm  
0     

In [30]:
# Merge Orders with Customers
order_customer = orders.merge(
    customers,
    on="customer_id",
    how="left"
)

print("Orders + Customers shape:", order_customer.shape)
order_customer.head()

Orders + Customers shape: (99441, 14)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay_days,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,7c396fd4830fd04220f754e42b4e5bff,3149,Sao Paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13,-6,af07308b275d755c9edb36a90c618231,47813,Barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,-18,3a653a41f6f9fc3d2a113cf8398680e8,75265,Vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,-13,7c142cf63193a1473d2e66489a9ae977,59296,Sao Goncalo Do Amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,-10,72632f0f9dd73dfee390c9b22eb56dd6,9195,Santo Andre,SP


In [31]:
# Merge with order items
order_customer_items = order_customer.merge(
    items,
    on="order_id",
    how="left"
)

print("After adding items:", order_customer_items.shape)
order_customer_items.head()

After adding items: (113425, 20)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay_days,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,7c396fd4830fd04220f754e42b4e5bff,3149,Sao Paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13,-6,af07308b275d755c9edb36a90c618231,47813,Barreiras,BA,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,-18,3a653a41f6f9fc3d2a113cf8398680e8,75265,Vianopolis,GO,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,-13,7c142cf63193a1473d2e66489a9ae977,59296,Sao Goncalo Do Amarante,RN,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,-10,72632f0f9dd73dfee390c9b22eb56dd6,9195,Santo Andre,SP,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [32]:
# Merge with products
order_customer_products = order_customer_items.merge(
    products,
    on="product_id",
    how="left"
)

print("After adding products:", order_customer_products.shape)
order_customer_products.head()

After adding products: (113425, 31)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay_days,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_length_norm,product_height_norm,product_width_norm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,40.0,268.0,4.0,500.0,19.0,8.0,13.0,0.122449,0.058252,0.062500
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13,-6,...,29.0,178.0,1.0,400.0,19.0,13.0,19.0,0.122449,0.106796,0.116071
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,-18,...,46.0,232.0,1.0,420.0,24.0,19.0,21.0,0.173469,0.165049,0.133929
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,-13,...,59.0,468.0,3.0,450.0,30.0,10.0,20.0,0.234694,0.077670,0.125000
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,-10,...,38.0,316.0,4.0,250.0,51.0,15.0,15.0,0.448980,0.126214,0.080357


In [33]:
# Merge with payments
master_df = order_customer_products.merge(
    payments,
    on="order_id",
    how="left"
)

print("Final enriched dataframe shape:", master_df.shape)
master_df.head()

Final enriched dataframe shape: (118428, 36)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay_days,...,product_height_cm,product_width_cm,product_length_norm,product_height_norm,product_width_norm,payment_sequential,payment_type,payment_installments,payment_value,payment_value_norm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,8.0,13.0,0.122449,0.058252,0.062500,1.0,credit_card,1.0,18.12,0.001325
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,8.0,13.0,0.122449,0.058252,0.062500,3.0,voucher,1.0,2.00,0.000146
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,8.0,13.0,0.122449,0.058252,0.062500,2.0,voucher,1.0,18.59,0.001360
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13,-6,...,13.0,19.0,0.122449,0.106796,0.116071,1.0,boleto,1.0,141.46,0.010352
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,-18,...,19.0,21.0,0.173469,0.165049,0.133929,1.0,credit_card,3.0,179.12,0.013108


In [34]:
print("Columns in final dataset:\n", master_df.columns)
print("\nSample rows:")
master_df.head()

Columns in final dataset:
 Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'delivery_time_days', 'delivery_delay_days', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'product_length_norm',
       'product_height_norm', 'product_width_norm', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value',
       'payment_value_norm'],
      dtype='object')

Sample rows:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delivery_delay_days,...,product_height_cm,product_width_cm,product_length_norm,product_height_norm,product_width_norm,payment_sequential,payment_type,payment_installments,payment_value,payment_value_norm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,8.0,13.0,0.122449,0.058252,0.062500,1.0,credit_card,1.0,18.12,0.001325
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,8.0,13.0,0.122449,0.058252,0.062500,3.0,voucher,1.0,2.00,0.000146
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,-8,...,8.0,13.0,0.122449,0.058252,0.062500,2.0,voucher,1.0,18.59,0.001360
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13,-6,...,13.0,19.0,0.122449,0.106796,0.116071,1.0,boleto,1.0,141.46,0.010352
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,-18,...,19.0,21.0,0.173469,0.165049,0.133929,1.0,credit_card,3.0,179.12,0.013108


In [36]:
master_df.to_csv("../enriched_ecommerce_data.csv", index=False)
print("Enriched dataset saved successfully!")

Enriched dataset saved successfully!
